<a href="https://colab.research.google.com/github/bckim9489/agent_practice/blob/main/agent_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Common Setup

참고 : 아래 Secret으로 Key 등록해서 실습함
1.   LLM API Key (GPT, claude 등) : OPENAI-KEY 로 등록했음
2.   Tavily Serach API Key (https://app.tavily.com/home) : TAVIL-KEY 로 등록했음



In [1]:
!pip -q install -U "requests==2.32.5" "pydantic==2.12.3" "httpx==0.28.1"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip -q install -U langgraph langchain-openai langchain-community wikipedia numexpr tavily-python

In [3]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata

# LLM API Key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI-KEY")

# Tools
# Search API Key
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY-KEY")

llm = ChatOpenAI(model="gpt-4o", temperature=0)


## ㄴ 0. TEST

In [6]:
import requests, httpx, pydantic
import langchain_community, langgraph
from langgraph.graph import StateGraph

print("requests:", requests.__version__)
print("httpx:", httpx.__version__)
print("pydantic:", pydantic.__version__)
print("langchain-community:", langchain_community.__version__)
print("LangGraph import OK")
print("StateGraph:", StateGraph)

requests: 2.32.5
httpx: 0.28.1
pydantic: 2.12.3
langchain-community: 0.4.1
LangGraph import OK
StateGraph: <class 'langgraph.graph.state.StateGraph'>


## ㄴ 1. Tools

In [7]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_core.tools import tool
import numexpr

# Wikipedia tool
wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=2000)
)

# 기존 wiki wrapper 재사용
wiki_lookup_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=4000)

@tool
def docstore_lookup(title: str) -> str:
    """
    Lookup a specific document by title from the docstore (Wikipedia).
    Input should be a precise title or entity name.
    """
    return wiki_lookup_wrapper.run(title)

# Tavily Search tool
search_tool = TavilySearchResults(max_results=5)

# Calculator tool (llm-math 대체)
@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression.
    Input must be a valid expression like '345*872' or '12/4+9'.
    """
    try:
        return str(numexpr.evaluate(expression))
    except Exception as e:
        return f"Error: {e}"


tools = [wiki, calculator, search_tool, docstore_lookup]
print("=== Tools ===")
for t in tools:
    print("-", t.name)

=== Tools ===
- wikipedia
- calculator
- tavily_search_results_json
- docstore_lookup


/tmp/ipython-input-1012767205.py:25: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=5)


#
----------------------------------------------------------

# [Gen 1] Prompt-driven Tool-Using Agents(ReAct-based Agents)
> **LLM**이 매 순간 다음 행동을 **즉흥 결정**
*   **LLM-Driven Agents**
*   LLM이 생각(Reasoning) 하고
*   외부 도구를 행동(Act) 으로 호출하며
*   그 결과를 다시 보고 추론을 이어가는 "Tool-using LLM Loop"
*   2022 ~ 2024




## ㄴ 1. Zero-Shot ReAct
*   기억 없음(Stateless)
*   **LLM + Tools**





### ㄴ 1.1 Description

#### ㄴ 1.1.1 Workflow


```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
LLM: 질문 해석 (Thought / Reasoning)
  ↓
Tool 필요 여부 판단
  ├─ 필요 없음 → 바로 답 생성
  └─ 필요 있음 → Tool 선택 (Action)
                      ↓
                Tool 실행 (Python / API / DB 등)
                      ↓
                결과 반환 (Observation)
                      ↓
LLM이 결과 반영하여 다시 Reasoning
  ↓
(필요 시 Thought → Action → Observation 반복)
  ↓
Final Answer 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시

```

#### ㄴ 1.1.2 Inner ReAct Loop



``` javascript
while 문제 해결 전:
    Thought      → 어떻게 풀지 스스로 판단
    Action       → 사용할 Tool 선택
    Observation  → Tool 실행 결과 받기
    Thought      → 결과 보고 다음 행동 결정
```



### ㄴ 1.2 Code

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

In [ ]:
q = """위키피디아에서 찾기 좋은 '검색어(제목형 키워드)'를 3개 만들어서,
그 키워드로 Wikipedia tool을 사용해 검색하고 근거를 요약해줘.
주제: Pinus densiflora 옮겨심기(이식) 적기"""

In [ ]:
result = agent.invoke({"messages": [("user", q)]})

print(result["messages"][-1].content)

### ㄴ 1.3 Verbose

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

for step in agent.stream(
    {"messages": [("user", q)]},
    stream_mode="values",
):
    msg = step["messages"][-1]

    # LLM이 생각/결정한 내용
    if isinstance(msg, AIMessage):
        print("\n AI MESSAGE")
        print(msg.content)

        if getattr(msg, "tool_calls", None):
            print(" TOOL CALL:", msg.tool_calls)

    # Tool 실행 결과
    elif isinstance(msg, ToolMessage):
        print("\n TOOL RESULT")
        print("tool:", msg.name)
        print(msg.content[:1000])  # 너무 길면 앞부분만


## ㄴ 2. Conversational ReAct
*   메모리 사용
*   **LLM + Tools + Checkpointer**



### ㄴ 2.1 Description

#### ㄴ 2.1.1 Workflow

```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
이전 대화 기록(Memory / State) 로드
  ↓
LLM: 현재 질문 + 대화 히스토리 함께 해석
  ↓
사용자의 의도 파악 (맥락 기반 Reasoning)
  ↓
Tool 필요 여부 판단
  ├─ 필요 없음 → 바로 답 생성
  └─ 필요 있음 → Tool 선택 (Action)
                      ↓
                Tool 실행 (Python / API / DB 등)
                      ↓
                결과 반환 (Observation)
                      ↓
LLM이 결과 + 이전 대화 맥락을 반영하여 다시 Reasoning
  ↓
(필요 시 Thought → Action → Observation 반복)
  ↓
Final Answer 생성
  ↓
대화 Memory(State)에 이번 메시지 저장
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시
```

#### ㄴ 2.1.2 Inner ReAct Loop

```javascript
while 문제 해결 전:
    Load Memory        → 이전 대화 불러오기
    Thought            → 맥락 기반으로 판단
    Action             → Tool 필요하면 실행
    Observation        → Tool 결과 받기
    Update Memory      → 새 정보 저장
```

### ㄴ 2.2 Code

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()
agent = create_react_agent(llm, tools, checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "chat-001"}}
q1 = "내 출생년도는 1994년이야. 지금은 계산하지 말고 이 정보만 기억해."
q2 = "그럼 내가 2026년에 몇 살인지 계산기를 꼭 사용해서 나이를 계산해서 알려줘"
q3 = "올해는 무슨 해이며 나는 무슨 띠인지 알려줘"

In [ ]:
result = agent.invoke({"messages": [("user", q1)]}, config=config)
print(result["messages"][-1].content)
result = agent.invoke({"messages": [("user", q2)]}, config=config)
print(result["messages"][-1].content)
result = agent.invoke({"messages": [("user", q3)]}, config=config)
print(result["messages"][-1].content)


### ㄴ 2.3 Verbose

#### ㄴ 2.3.1 Test Setup

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage
import time
from openai import RateLimitError

from langchain_core.messages import AIMessage, ToolMessage

def stream_with_retry(agent, payload, config, stream_mode="values", max_retries=6):
    """
    agent.stream(...) 를 RateLimitError(429) 발생 시 backoff 재시도.
    """
    attempt = 0
    while True:
        try:
            for step in agent.stream(payload, config=config, stream_mode=stream_mode):
                yield step
            return  # 정상 종료
        except RateLimitError as e:
            attempt += 1
            if attempt > max_retries:
                raise

            # 간단 backoff (점점 더 기다림)
            wait = min(20, 1.5 * attempt)
            print(f"\n RateLimit(429). retry in {wait:.1f}s ...")
            time.sleep(wait)



def run_verbose(question: str, label: str, thread_id: str):
    print("\n" + "="*20 + f" {label} (thread_id={thread_id}) " + "="*20)
    print("USER:", question)

    cfg = {"configurable": {"thread_id": thread_id}}

    payload = {"messages": [("user", question)]}

    last_ai = None
    for step in stream_with_retry(agent, payload, cfg, stream_mode="values"):
        msg = step["messages"][-1]

        if isinstance(msg, AIMessage):
            last_ai = msg
            if msg.content:
                print("\nAI:", msg.content)
            if getattr(msg, "tool_calls", None):
                print("tool_calls:", msg.tool_calls)

        elif isinstance(msg, ToolMessage):
            print("\nTOOL RESULT:", msg.name)
            print(msg.content[:800])

    return last_ai.content if last_ai else None

#메모리 사용하는지 A/B 테스트
def ab_test(q1: str, q2: str, q3:str, thread_a: str="mem-A", thread_b: str="mem-B", thread_c: str="mem-C"):
    print("\n\n" + "#"*10 + " A/B MEMORY TEST START " + "#"*10)

    # A: q1 using thread_id a
    run_verbose(q1, "A-1: seed memory (q1)", thread_a)


    # B: q2 using thread_id b
    run_verbose(q2, "B-1: q2 only (no memory)", thread_b)


    # B: q3 using thread_id c
    run_verbose(q3, "C-1: q3 only (no memory)", thread_c)

    print("\n" + "#"*10 + " A/B MEMORY TEST END " + "#"*10)


#### ㄴ 2.3.2 Test A (같은 thread_id)

*   Q1을 thread_id A 에 저장
*   Q2를 thread_id A 에 질의
----------------------------
| Q2에서 메모리에 저장한 Q1을 활용하여 답변할 것으로 예상

In [ ]:
# 같은 thread_id
run_verbose(q1, "Q1", "chat-001")
run_verbose(q2, "Q2", "chat-001")
run_verbose(q3, "Q3", "chat-001")

#### ㄴ 2.3.3 TEST B (다른 thread_id)

*   Q1을 thread_id A 에 저장
*   Q2를 thread_id B에 질의
----------------------------
| Q2에서 질의 에 대한 정보 요구 예상



In [ ]:
ab_test(q1, q2, q3, thread_a="chat-003", thread_b="chat-004", thread_c="chat-005")

## ㄴ 3. Search-augmented ReAct (Self-ask with search)

*   질문 분해 중심
*   **RAG와 매우 유사**
*   사실상 Self-Ask = 초기 형태의 RAG



### ㄴ 3.1 Description

#### ㄴ 3.1.1. Workflow



```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph / Agent Runtime 시작
  ↓
LLM이 질문을 "하위 질문(Sub-questions)"으로 분해
  ↓
각 하위 질문을 Search Tool로 순차 조회
  ↓
검색 결과(Observation)를 모아 중간 결론 생성
  ↓
필요하면 추가 하위 질문 생성 → 다시 Search
  ↓
모든 정보가 충분해지면 최종 답 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시
```



#### ㄴ 3.1.2 Inner ReAct Loop

```javascript
while 답을 만들 정보가 부족하면:
    Follow-up Question 생성
    Search Tool 실행
    Observation 수집
    현재까지의 사실 정리
```

### ㄴ 3.2 Code

In [ ]:
from langchain_core.messages import SystemMessage

# Self-Ask의 핵심 포맷을 강제
SELF_ASK_POLICY = SystemMessage(content="""
You are a Self-Ask-with-Search agent.

Hard rules:
- For EACH follow-up question, you MUST call the tavily_search_results_json tool.
- After the tool returns, you MUST write an Intermediate answer that uses the tool result.
- Intermediate answer must include 2-3 bullet points and mention the source titles (from tool results).
- Do NOT write placeholders like "...". If info is missing, do another search.

Output format (exact):
Follow-up 1: ...
Intermediate answer 1:
- ...
- ...
Sources: <title1>, <title2>

Follow-up 2: ...
Intermediate answer 2:
- ...
- ...
Sources: ...

Final answer: ...
""")


In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

In [ ]:
q = """AI Agent를 3문장으로 설명해줘.
단, 2025~2026년 기준으로 최신 경향을 반영하기 위해 반드시 tavily_search_results_json을 최소 2번 사용하고,
각 검색 결과를 근거로 Intermediate answer를 작성한 뒤 Final answer를 써."""

In [ ]:
config = {"configurable": {"thread_id": "self-ask-01"}}

result = agent.invoke({"messages": [SELF_ASK_POLICY, ("user", q)]},config=config)

print(result["messages"][-1].content)

### ㄴ 3.3. Verbose

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

def run_self_ask_verbose(question: str, thread_id="self-ask-03", tool_preview=600):
    cfg = {"configurable": {"thread_id": thread_id}}

    print("\n" + "="*80)
    print("USER:", question)
    print("="*80)

    for step in agent.stream(
        {"messages": [SELF_ASK_POLICY, ("user", question)]},
        config=cfg,
        stream_mode="values",
    ):
        msg = step["messages"][-1]

        # 1) 모델이 말한 텍스트(중간 출력 포함)
        if isinstance(msg, AIMessage):
            if msg.content:  # content가 있는 경우만 출력
                print("\n AI MESSAGE:\n", msg.content)

            # 2) 도구 호출 로그
            if getattr(msg, "tool_calls", None):
                print("\n TOOL CALLS:")
                for tc in msg.tool_calls:
                    print(" -", tc)

        # 3) 도구 결과
        elif isinstance(msg, ToolMessage):
            print("\n TOOL RESULT:", msg.name)
            print(msg.content[:tool_preview])

    print("\n" + "="*80)
    print("END")
    print("="*80)

run_self_ask_verbose(q)


## ㄴ 4. ReAct docstore

*   LLM이 문서 저장소를 스스로 탐색
*   필요한 문서를 조회(Lookup)하여 근거 기반 답변을 생성



### ㄴ 4.1 Description

#### ㄴ 4.1.1 Workflow

```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
LLM: 질문 해석 (문서 탐색 필요 판단)
  ↓
Docstore Search Tool 호출 (Search Action)
  ↓
관련 문서 후보 목록 반환 (Observation)
  ↓
LLM: 어떤 문서를 읽을지 Reasoning
  ↓
Docstore Lookup Tool 호출 (Lookup Action)
  ↓
선택된 문서 실제 내용 반환 (Observation)
  ↓
LLM이 문서 내용을 기반으로 재해석 / 근거 정리
  ↓
(필요 시 Search → Lookup 반복)
  ↓
충분한 근거 확보 후 Final Answer 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시

```

#### ㄴ 4.1.2 Inner ReAct Loop

```javascript
while (답변에 필요한 근거가 충분하지 않다):
    Thought: 질문을 해결하려면 어떤 문서를 찾아야 하는가?
    Action: Search(query)
    Observation: 관련 문서 후보 목록 반환
    Thought: 어떤 문서를 읽어야 하는가?
    Action: Lookup(document_id)
    Observation: 문서 내용 확보
    Thought:이 정보로 답변 가능한가?
            ├─ NO → 다른 문서 다시 Search
            └─ YES → 반복 종료
```

### ㄴ 4.2 Code

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

In [ ]:
q = """
규칙:
- wikipedia 도구를 1회 호출해서 후보 제목을 찾는다.
- 그 다음 docstore_lookup 도구를 반드시 1회 이상 호출한다.
- docstore_lookup 결과를 근거로 최종 답변을 작성한다.

질문: 밍크 선인장(Mammillaria)은 어떤 식물인지 설명해줘.
"""

In [ ]:

config = {"configurable": {"thread_id": "docstore-01"}}
result = agent.invoke({"messages": [("user", q)]},config=config)

print(result["messages"][-1].content)


### ㄴ 4.3 Verbose

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

def run_docstore_verbose(question: str, thread_id="docstore-debug"):
    cfg = {"configurable": {"thread_id": thread_id}}

    print("\n================ USER ================")
    print(question)

    for step in agent.stream(
        {"messages": [("user", question)]},
        config=cfg,
        stream_mode="values",   # 상태 변화 전부 받기
    ):
        msg = step["messages"][-1]

        # LLM이 생각한 내용 (Thought / Action 결정)
        if isinstance(msg, AIMessage):
            if msg.content:
                print("\n AI THOUGHT:")
                print(msg.content)

            if getattr(msg, "tool_calls", None):
                print("\n TOOL CALL:")
                print(msg.tool_calls)

        # Tool 실행 결과 (Observation)
        elif isinstance(msg, ToolMessage):
            print("\n TOOL RESULT:", msg.name)
            print(msg.content[:1000])  # 너무 길어서 제한

    print("\n================ END ================\n")

run_docstore_verbose(q)


#
--------------------------------------------------------------

# [Gen 2] LLM Orchestrated Systems(Post-ReAct Agent Architectures)
> Workflow Engine이 **순서를 결정**, LLM은 **필요할 때만 호출**
*   **System-Driven Agents**
*   계획 수립 / 의도 해석   ← LLM
*   실행 흐름은 시스템이 통제 ← Runtime / Graph / Process
*   Tool 실행은 강제된 구조로 수행
*   상태(State)를 외부에서 관리
*   2024 ~ 현재

```javascript
User → Orchestrator → Planner → Executor → Validator → State Update
                        ↓
                      LLM (as function)
```


## ㄴ 1. Plan-Then-Execute Core
> Plan → **Freeze** Plan → **Execute** Deterministically
>

### ㄴ 1.1 Description

```javascript
Planner LLM → Execution Graph 생성
Execution Engine:
    Step1
    Step2
    Step3
```
*    **Planner(LLM)** 는 계획(어떤 툴을 어떤 순서로 쓸지)를 JSON Plan으로만 만듬 (실행 금지)
*    **Executor(시스템)** 가 계획을 결정론적으로 실행 (툴을 실제로 호출)
*    **Writer(LLM)** 가 결과를 자연어로 정리 및 답변

```javascript
User Input
   ↓
Planner (LLM 1회)
   ↓
Executor (Deterministic Tool Calls)
   ↓
Writer (LLM 정리)
   ↓
End
```

### ㄴ 1.2 Code

#### ㄴ 1.2.1 Tool Registry

In [8]:
import json
from typing import List, Literal, Dict, Any

# 우리가 Plan에서 쓸 "툴 alias"
TOOL_REGISTRY = {
    "wiki": wiki,
    "search": search_tool,
    "lookup": docstore_lookup,
    "calc": calculator,
}

ToolAlias = Literal["wiki", "search", "lookup", "calc", "final_answer"]

#### ㄴ 1.2.2 Contract (Plan Schema)

In [9]:
from pydantic import BaseModel, Field

class Step(BaseModel):
    tool: ToolAlias
    input: str

class Plan(BaseModel):
    goal: str
    steps: List[Step] = Field(min_items=1, max_items=6)

/tmp/ipython-input-3491100315.py:9: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  steps: List[Step] = Field(min_items=1, max_items=6)
/tmp/ipython-input-3491100315.py:9: PydanticDeprecatedSince20: `max_items` is deprecated and will be removed, use `max_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  steps: List[Step] = Field(min_items=1, max_items=6)


#### ㄴ 1.2.3 System Prompt

##### ㄴ 1.2.3.1 Planner System Prompt

In [10]:
from langchain_core.messages import SystemMessage, HumanMessage

PLANNER_SYS = SystemMessage(content=
"""You are a Planner.
Return ONLY valid JSON matching this schema:

{
  "goal": string,
  "steps": [{"tool": "wiki"|"search"|"lookup"|"calc"|"final_answer", "input": string}, ...]
}

Rules:
- Do NOT execute tools.
- Prefer "search" for up-to-date web info when needed.
- Use "wiki" for general grounding.
- Use "lookup" only when you know the exact Wikipedia title/entity.
- Use "calc" only for arithmetic.
- Keep steps minimal (<=4 if possible).
- The last step MUST be "final_answer".
""")

##### ㄴ 1.2.3.2 Writer System Prompt

In [106]:
WRITER_SYS = SystemMessage(content=
"""You are a Writer.

You will receive:
- user_question
- plan (JSON)
- tool_results (JSON)

Rules:
- Write the final answer in Korean only (한국어로만 작성).
- Use tool_results as evidence. If you reference web search results, summarize them in Korean.
- Keep it concise and structured (bullets ok).
- If any tool output contains "Error" or "ERROR", explain briefly in Korean and propose a corrected plan.

Return ONLY JSON:
{"final": string}
""")


#### ㄴ 1.2.4 Planner Function

In [26]:
import re, json

def extract_json(text: str) -> str:
    """LLM 응답에서 JSON만 뽑아낸다 (```json``` 코드블록/앞뒤 잡문 방어)."""
    if text is None:
        return ""
    t = text.strip()

    # 1) ```json ... ``` 블록 우선 추출
    m = re.search(r"```(?:json)?\s*(\{.*\})\s*```", t, flags=re.DOTALL | re.IGNORECASE)
    if m:
        return m.group(1).strip()

    # 2) 첫 { 부터 마지막 } 까지 잘라내기
    start = t.find("{")
    end = t.rfind("}")
    if start != -1 and end != -1 and end > start:
        return t[start:end+1].strip()

    return t


def make_plan(user_question: str) -> Plan:
    resp = llm.invoke([PLANNER_SYS, HumanMessage(content=user_question)])
    json_str = extract_json(resp.content)
    data = json.loads(resp.content)
    return Plan.model_validate(data)

#### ㄴ 1.2.5 Executor Function

In [14]:
def _invoke_tool(alias: str, tool_input: str) -> Any:
    tool = TOOL_REGISTRY[alias]

    # TavilySearchResults는 보통 {"query": "..."} 형태가 안정적
    if alias == "search":
        try:
            return tool.invoke({"query": tool_input})
        except Exception:
            return tool.invoke(tool_input)

    # 나머지는 보통 문자열 입력
    try:
        return tool.invoke(tool_input)
    except Exception:
        return tool.run(tool_input)

def execute_plan(plan: Plan) -> List[Dict[str, Any]]:
    results = []
    for idx, step in enumerate(plan.steps):
        if step.tool == "final_answer":
            results.append({
                "step_index": idx,
                "tool": step.tool,
                "input": step.input,
                "output": "(final step - no tool execution)"
            })
            continue

        try:
            out = _invoke_tool(step.tool, step.input)
        except Exception as e:
            out = f"ERROR: {type(e).__name__}: {e}"

        results.append({
            "step_index": idx,
            "tool": step.tool,
            "input": step.input,
            "output": out
        })
    return results

#### ㄴ 1.2.6 Writer Function

In [15]:
def write_final(user_question: str, plan: Plan, tool_results: List[Dict[str, Any]]) -> str:
    payload = {
        "user_question": user_question,
        "plan": plan.model_dump(),
        "tool_results": tool_results,
    }
    resp = llm.invoke([WRITER_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))])
    data = json.loads(resp.content)
    return data["final"]

#### ㄴ 1.2.7 User Prompt

In [17]:
question = "Plan-Then-Execute 패턴이 뭐고 ReAct랑 차이를 예시 포함해서 설명해줘. 최신 관점이면 search도 활용해."

#### ㄴ 1.2.8 Result

In [107]:
def run_plan_then_execute(question: str) -> str:
    # 1) Plan 생성
    plan = make_plan(question)

    # 2) 계획 실행 (결정론적)
    tool_results = execute_plan(plan)

    # 3) 최종 답변 생성
    final = write_final(question, plan, tool_results)

    return final

answer = run_plan_then_execute(question)
print(answer)

State Machine 실행 모델은 유한 상태 기계(FSM)로, 특정 입력이나 이벤트에 따라 상태 간 전환을 정의하는 수학적 모델입니다. FSM은 한 번에 하나의 상태에만 있을 수 있으며, 입력에 따라 다른 상태로 전환됩니다. 예를 들어, 자판기나 엘리베이터, 신호등 등이 FSM의 예입니다.

Plan-Then-Execute 모델은 목표를 달성하기 위한 상세한 계획을 먼저 수립한 후, 그 계획을 실행하는 방식입니다. 예를 들어, 로봇이 물체를 이동시키는 작업을 수행할 때, FSM을 사용하여 '대기', '이동', '집기', '놓기'와 같은 상태를 정의할 수 있습니다. Plan-Then-Execute 접근법을 사용하면, 로봇은 먼저 물체를 한 위치에서 다른 위치로 이동시키기 위한 일련의 행동 계획을 세우고, 그 후 각 상태를 단계별로 전환하며 계획을 실행합니다.


#### ㄴ 1.2.9 Debuging

In [19]:
def debug_make_plan(user_question: str):
    resp = llm.invoke([PLANNER_SYS, HumanMessage(content=user_question)])
    print("=== RAW PLANNER OUTPUT ===")
    print(repr(resp.content))
    return resp.content

def debug_write_final(user_question: str, plan, tool_results):
    payload = {
        "user_question": user_question,
        "plan": plan.model_dump(),
        "tool_results": tool_results,
    }
    resp = llm.invoke([WRITER_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))])
    print("=== RAW WRITER OUTPUT ===")
    print(repr(resp.content))
    return resp.content


In [24]:
question = "Plan-Then-Execute 패턴이 뭐고 ReAct랑 차이를 예시 포함해서 설명해줘. 최신 관점이면 search도 활용해."

plan = make_plan(question)
tool_results = execute_plan(plan)

raw_writer = debug_write_final(question, plan, tool_results)

=== RAW WRITER OUTPUT ===
'{"final": "The Plan-Then-Execute pattern and the ReAct pattern are two distinct approaches used in AI and software development, particularly for managing tasks and processes.\\n\\n1. **Plan-Then-Execute Pattern**: This approach involves a clear separation between planning and execution phases. Initially, a comprehensive plan is developed, breaking down tasks into subtasks and creating a detailed execution strategy. Once the plan is set, the execution phase follows, where tasks are carried out in sequence. This pattern is well-suited for complex, multi-step tasks that require strategic planning and coordination, as it allows for a structured and predictable approach. An example could be a project management tool that first outlines all project phases before any work begins.\\n\\n2. **ReAct Pattern**: The ReAct (Reason-Act) pattern operates in a tight, iterative loop where the system continuously alternates between reasoning (thinking about the next step) and a

### ㄴ 1.3 Verbose

In [31]:
def run_plan_then_execute_verbose(user_question: str, print_chars: int = 800):
    print("\n" + "="*80)
    print("USER QUESTION:")
    print(user_question)

    # PLAN 단계
    print("\n" + "-"*30 + " PLANNING " + "-"*30)

    plan = make_plan(user_question)

    print("GOAL:", plan.goal)
    print("\nSTEPS:")
    for i, step in enumerate(plan.steps):
        print(f"  [{i}] tool={step.tool} | input={step.input}")

    # EXECUTE 단계
    print("\n" + "-"*30 + " EXECUTION " + "-"*30)

    tool_results = execute_plan(plan)

    for r in tool_results:
        print(f"\n[STEP {r['step_index']}] TOOL: {r['tool']}")
        print("INPUT:", r["input"])

        out = r["output"]
        if not isinstance(out, str):
            out = json.dumps(out, ensure_ascii=False)

        print("OUTPUT (truncated):")
        print(out[:print_chars])

    # WRITE 단계
    print("\n" + "-"*30 + " WRITING FINAL ANSWER " + "-"*30)

    final = write_final(user_question, plan, tool_results)

    print("\nFINAL ANSWER:")
    print(final)

    print("\n" + "="*80)

    return {
        "plan": plan,
        "tool_results": tool_results,
        "final": final
    }


In [32]:
run_plan_then_execute_verbose(question)


USER QUESTION:
Plan-Then-Execute 패턴이 뭐고 ReAct랑 차이를 예시 포함해서 설명해줘. 최신 관점이면 search도 활용해.

------------------------------ PLANNING ------------------------------
GOAL: Explain the Plan-Then-Execute pattern and its differences from ReAct, including examples.

STEPS:
  [0] tool=wiki | input=Plan-Then-Execute pattern
  [1] tool=wiki | input=ReAct pattern
  [2] tool=search | input=Plan-Then-Execute vs ReAct pattern 2023
  [3] tool=final_answer | input=Provide a detailed explanation of the Plan-Then-Execute pattern and how it differs from the ReAct pattern, including examples from the latest sources.

------------------------------ EXECUTION ------------------------------

[STEP 0] TOOL: wiki
INPUT: Plan-Then-Execute pattern
OUTPUT (truncated):
Page: Strategic planning
Summary: Strategic planning or corporate planning is an activity undertaken by an organization through which it seeks to define its future direction and makes decisions such as resource allocation aimed at achieving its intended

{'plan': Plan(goal='Explain the Plan-Then-Execute pattern and its differences from ReAct, including examples.', steps=[Step(tool='wiki', input='Plan-Then-Execute pattern'), Step(tool='wiki', input='ReAct pattern'), Step(tool='search', input='Plan-Then-Execute vs ReAct pattern 2023'), Step(tool='final_answer', input='Provide a detailed explanation of the Plan-Then-Execute pattern and how it differs from the ReAct pattern, including examples from the latest sources.')]),
 'tool_results': [{'step_index': 0,
   'tool': 'wiki',
   'input': 'Plan-Then-Execute pattern',
   'output': 'Page: Strategic planning\nSummary: Strategic planning or corporate planning is an activity undertaken by an organization through which it seeks to define its future direction and makes decisions such as resource allocation aimed at achieving its intended goals. "Strategy" has many definitions, but it generally involves setting major goals, determining actions to achieve these goals, setting a timeline, and mobili

## ㄴ 2. State Machine

*    LLM은 **상태를 읽고 판단만** 함.
*    LLM은 이제 **행동 주체**가 아니라 **State**를 평가하는 **Decision Function**으로만 쓰임


***Plan-Then-Execute 코드는 그대로 재사용***

### ㄴ 2.1 Description

> **Plan-Then-Execute**는 **'무엇을 할지'** 정의한 패턴이고, <br>
**State Machine**은 **'그걸 어떻게 실행할지'** 를 정의하는 실행 모델이다.

**Plan-Then-Execute**와 **State Machine**의 차이
| Plan-Then-Execute       | State Machine                      |
| ------------ | -------------------------- |
| 함수가 흐름을 결정   | 상태가 흐름을 결정                 |
| 코드가 다음 단계 호출 | Transition Logic이 다음 상태 선택 |
| LLM = 작업 수행자 | LLM = 상태 판단자               |

<br>

**Plan-Then-Execute**에서의 역할이 **State Machine** 에서 아래와 같이 역할이 재배치됨

| 구성요소     | State Machine에서 역할     |
| -------- | ---------------------- |
| Planner  | `PLAN` 상태에서 호출되는 액션    |
| Executor | `EXECUTE` 상태에서 호출되는 액션 |
| Writer   | `WRITE` 상태에서 호출되는 액션   |




```javascript
TaskState:
    goal
    progress
    artifacts
    failures
```



### ㄴ 2.1 Code

#### ㄴ 2.1.0 Utils

##### ㄴ 2.1.0.1 Replan 전용 Planner 래퍼

In [99]:
from langchain_core.messages import SystemMessage, HumanMessage
import json

# (옵션) Replan 상황에서만 사용할 system prompt (기존 PLANNER_SYS를 안 건드리려면 별도 생성)
REPLAN_PLANNER_SYS = SystemMessage(content=
"""You are a Planner for an LLM agent system.

You MUST output ONLY valid JSON that matches this schema:
{
  "goal": string,
  "steps": [
    {"tool": "wiki|search|calculator|docstore_lookup|final_answer", "input": string}
  ]
}

Rules:
- Do NOT use markdown fences.
- If the user asks for an example, include a step that gathers info (wiki/search) AND ensure the final_answer step requests a concrete example with explicit state transitions.
- Prefer search over wiki when the term is ambiguous or not a Wikipedia entry (e.g., Plan-and-Execute in LLM agents).

Return ONLY JSON.
""")

# structured output으로 스키마 강제
replan_planner_llm = llm.with_structured_output(Plan)

def make_plan_replan(user_question: str, fix_hint: str | None = None) -> Plan:
    """
    Replan 전용 Planner.
    - fix_hint를 별도 채널로 주어 plan 스키마가 깨질 확률을 낮춤
    - structured_output(Plan)로 강제
    """
    payload = {
        "user_question": user_question,
        "fix_hint": fix_hint or ""
    }
    return replan_planner_llm.invoke([
        REPLAN_PLANNER_SYS,
        HumanMessage(content=json.dumps(payload, ensure_ascii=False))
    ])


##### ㄴ 2.1.0.2 State Machine 전용 Writer 래퍼

In [112]:
from langchain_core.messages import SystemMessage, HumanMessage
import json

STATE_WRITER_SYS = SystemMessage(content=
"""You are a Writer that produces the final answer for the user in Korean.

You receive a JSON payload with:
- user_question
- plan
- tool_results

Write a clear answer that MUST include:
1) 정의: Plan-Then-Execute와 State Machine 실행 모델의 관계
2) 차이: 단순 파이프라인(plan→execute→write) vs 상태기반 전이(phase-driven)
3) 구체 예시 1개 (연구/로봇/업무 자동화 중 아무거나)
4) 상태 전이 로그(한 줄): 순서대로 어떻게 되는지

Constraints:
- Keep it practical and concrete.
- Do not mention internal tool names.
- Output plain text (no JSON), Korean only.
""")

def write_final_state_machine(user_question: str, plan, tool_results) -> str:
    payload = {
        "user_question": user_question,
        "plan": plan.model_dump(),
        "tool_results": tool_results,
    }
    resp = llm.invoke([
        STATE_WRITER_SYS,
        HumanMessage(content=json.dumps(payload, ensure_ascii=False))
    ])
    return (resp.content or "").strip()


#### ㄴ 2.1.1 State Definition

In [79]:
from typing import TypedDict, Literal, List, Dict, Any, Optional

Phase = Literal["PLAN", "EXECUTE", "VALIDATE", "WRITE", "DONE", "ERROR"]

class AgentState(TypedDict, total=False):
    # inputs
    user_question: str

    # state machine
    phase: Phase
    error: Optional[str]

    # artifacts (persisted across phases)
    plan: Any                 # Plan (pydantic model)
    tool_results: List[Dict[str, Any]]
    final: str

    # replan control
    replan_count: int
    max_replans: int

    fix_hint: str

    # debugging / tracing
    trace: List[Dict[str, Any]]

##### ㄴ 2.1.2 Validate

###### ㄴ 2.1.2.1 Contract (Validate Schema)

In [80]:
from pydantic import BaseModel
from langchain_core.messages import SystemMessage, HumanMessage
import json

class ValidationOut(BaseModel):
    ok: bool
    reason: str
    fix_hint: str  # replan을 한다면 planner에게 줄 힌트(짧게)

###### ㄴ 2.1.2.2 Validate System Prompt

In [101]:
VALIDATOR_SYS = SystemMessage(content=
"""You are a Validator for an LLM agent.

Input JSON contains:
- user_question
- plan
- tool_results

Decide if the information is sufficient to write a good answer.

Mark ok=false ONLY if:
- the retrieved content is clearly about the wrong concept (e.g., database query plan),
- or there is no material to produce at least one concrete example and a state-transition trace.

Otherwise ok=true.

Return ONLY JSON:
{"ok": boolean, "reason": string (Korean), "fix_hint": string (Korean)}
""")

#### ㄴ 2.1.3 Actions
> ***Plan-Then-Execute에서 사용한 함수 재사용***

##### ㄴ 2.1.3.1 Plan Action

In [95]:
def action_plan(state: AgentState) -> AgentState:
    q = state["user_question"]
    fix_hint = (state.get("fix_hint") or "").strip()

    try:
        # replan일 때만 replan planner 사용
        if fix_hint:
            plan = make_plan_replan(q, fix_hint)
            used_replan_planner = True
        else:
            plan = make_plan(q)  # 기존 core planner
            used_replan_planner = False

        state["plan"] = plan
        state["trace"].append({
            "phase": "PLAN",
            "ok": True,
            "used_replan_planner": used_replan_planner,
            "steps": [s.model_dump() for s in plan.steps],
        })

        state["phase"] = "EXECUTE"
        state["fix_hint"] = ""  # 힌트 소모
        return state

    except Exception as e:
        state["error"] = f"PLAN_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase": "PLAN", "ok": False, "error": state["error"]})
        state["phase"] = "ERROR"
        return state


##### ㄴ 2.1.3.2 Execute Action

In [83]:
def action_execute(state: AgentState) -> AgentState:
    try:
        plan = state["plan"]
        tool_results = execute_plan(plan)  # Plan-Then-Execute 에서의 Executer 함수
        state["tool_results"] = tool_results
        state["trace"].append({"phase":"EXECUTE", "ok": True, "num_results": len(tool_results)})
        state["phase"] = "VALIDATE"
    except Exception as e:
        state["error"] = f"EXECUTE_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase":"EXECUTE", "ok": False, "error": state["error"]})
        state["phase"] = "ERROR"
    return state

##### ㄴ 2.1.3.3 Write Action

In [109]:
def action_write(state: AgentState) -> AgentState:
    try:
        q = state["user_question"]
        plan = state["plan"]
        tool_results = state.get("tool_results", [])
        final = write_final_state_machine(q, plan, tool_results)
        state["final"] = final
        state["trace"].append({"phase":"WRITE", "ok": True, "final_chars": len(final)})
        state["phase"] = "DONE"
    except Exception as e:
        state["error"] = f"WRITE_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase":"WRITE", "ok": False, "error": state["error"]})
        state["phase"] = "ERROR"
    return state

##### ㄴ 2.1.3.4 Validate Action

In [85]:
import re

def action_validate(state: AgentState) -> AgentState:
    try:
        payload = {
            "user_question": state["user_question"],
            "plan": state["plan"].model_dump() if "plan" in state else None,
            "tool_results": state.get("tool_results", []),
        }
        # 구조화 출력(권장): JSON 깨짐 방지
        validator_llm = llm.with_structured_output(ValidationOut)

        verdict = validator_llm.invoke([
            VALIDATOR_SYS,
            HumanMessage(content=json.dumps(payload, ensure_ascii=False))
        ])

        state["trace"].append({
            "phase": "VALIDATE",
            "ok": True,
            "verdict": verdict.model_dump()
        })

        if verdict.ok:
            state["phase"] = "WRITE"
        else:
            # replan 제한
            state["replan_count"] = state.get("replan_count", 0) + 1
            if state["replan_count"] > state.get("max_replans", 2):
                state["error"] = f"VALIDATION_FAILED_MAX_REPLANS: {verdict.reason}"
                state["phase"] = "ERROR"
            else:
                # planner에게 힌트 남겨두기
                state["fix_hint"] = verdict.fix_hint
                state["trace"].append({
                    "phase": "REPLAN_TRIGGER",
                    "replan_count": state["replan_count"],
                    "reason": verdict.reason,
                    "fix_hint": verdict.fix_hint
                })
                # 힌트를 질문에 덧붙여 재계획 유도 (가장 단순한 방식)
                state["user_question"] = (
                    state["user_question"]
                    + "\n\n[Validator Hint]\n"
                    + verdict.fix_hint
                )
                state["phase"] = "PLAN"

    except Exception as e:
        state["error"] = f"VALIDATE_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase":"VALIDATE", "ok": False, "error": state["error"]})
        state["phase"] = "ERROR"

    return state


#### ㄴ 2.1.4 Transition Function (상태 전이 로직)

In [86]:
def transition(state: AgentState) -> AgentState:
    phase = state["phase"]

    if phase == "PLAN":
        return action_plan(state)
    elif phase == "EXECUTE":
        return action_execute(state)
    elif phase == "VALIDATE":
        return action_validate(state)
    elif phase == "WRITE":
        return action_write(state)
    elif phase in ("DONE", "ERROR"):
        return state
    else:
        state["error"] = f"UNKNOWN_PHASE: {phase}"
        state["phase"] = "ERROR"
        return state

#### ㄴ 2.1.4 User Prompt

In [87]:
question = "State Machine 실행 모델로 Plan-Then-Execute를 돌린다는 게 무슨 의미인지 예시로 설명해줘."

#### ㄴ 2.1.4 Result

In [102]:
def run_state_machine(question: str) -> str:
    state: AgentState = {
        "user_question": question,
        "phase": "PLAN",
        "error": None,
        "trace": [],
        "replan_count": 0,
        "max_replans": 2,
        "fix_hint": "",
    }

    while state["phase"] not in ("DONE", "ERROR"):
        state = transition(state)

    if state["phase"] == "DONE":
        return state["final"]

    raise RuntimeError(state.get("error", "Unknown error"))

In [113]:
print(run_state_machine(question))

1) **정의**: Plan-Then-Execute 모델은 고수준의 전략적 계획과 로컬화된 전술적 실행을 분리하는 아키텍처 패러다임입니다. 이 모델은 상태 기계(State Machine)와 결합하여, 계획 단계에서 전체 작업을 위한 고정된 순서의 도구 호출을 생성하고, 실행 단계에서 이 순서를 따라 작업을 수행합니다. 상태 기계는 각 단계의 상태를 추적하고, 필요에 따라 상태 전이를 통해 다음 단계로 넘어갑니다.

2) **차이**: 단순 파이프라인 방식에서는 계획(Plan) → 실행(Execute) → 결과 작성(Write) 순서로 일방향으로 진행됩니다. 반면, 상태기반 전이 방식에서는 각 단계가 상태로 정의되고, 상태 전이를 통해 다음 단계로 이동합니다. 이는 각 상태에서의 조건에 따라 유연하게 전환할 수 있어 복잡한 작업에 적합합니다.

3) **구체 예시**: 로봇 청소기의 경우를 생각해봅시다. 로봇 청소기는 방의 지도를 작성(Plan)하고, 그에 따라 청소 경로를 설정합니다. 이후 실행 단계에서 로봇은 설정된 경로를 따라 이동하며 청소를 수행합니다. 이 과정에서 장애물을 만나면 상태 기계가 이를 감지하고, 장애물을 피하는 새로운 경로로 상태를 전이시킵니다.

4) **상태 전이 로그**: 
   - 초기 상태: 지도 작성
   - 상태 전이: 경로 설정
   - 상태 전이: 청소 시작
   - 상태 전이: 장애물 감지
   - 상태 전이: 경로 수정
   - 상태 전이: 청소 재개
   - 종료 상태: 청소 완료

이와 같이 상태 기계는 각 단계의 상태를 관리하며, 필요에 따라 상태를 전이시켜 작업을 유연하게 수행합니다.


### ㄴ 2.2 Verbose

In [114]:
import json

def run_state_machine_verbose(question: str, print_chars: int = 600, max_steps: int = 20) -> AgentState:
    state: AgentState = {
        "user_question": question,
        "phase": "PLAN",
        "error": None,
        "trace": [],
        "replan_count": 0,
        "max_replans": 2,
        "fix_hint": "",
    }

    print("\n" + "="*80)
    print("USER QUESTION:")
    print(question)

    steps = 0
    while state["phase"] not in ("DONE", "ERROR"):
        steps += 1
        if steps > max_steps:
            state["error"] = "MAX_STEPS_EXCEEDED"
            state["phase"] = "ERROR"
            break

        print("\n" + "-"*28 + f" PHASE: {state['phase']} " + "-"*28)
        state = transition(state)

        if state["phase"] == "EXECUTE" and "plan" in state:
            print("Plan ready. Steps:")
            for i, s in enumerate(state["plan"].steps):
                print(f"  [{i}] {s.tool} :: {s.input}")

        if state["phase"] == "VALIDATE":
            print(f"Executed {len(state.get('tool_results', []))} steps. (Next: VALIDATE)")

        if state["phase"] == "PLAN" and state.get("replan_count", 0) > 0:
            print(f"Replanning... (count={state['replan_count']}/{state['max_replans']})")

        if state["phase"] == "WRITE":
            print("Validation passed. Moving to WRITE.")

    print("\n" + "-"*28 + f" PHASE: {state['phase']} " + "-"*28)

    if state["phase"] == "DONE":
        print("\nFINAL:")
        print(state["final"])
    else:
        print("\nERROR:")
        print(state.get("error"))

    print("\nTRACE (last 6):")
    trace = state.get("trace", [])
    print(json.dumps(trace[-6:], ensure_ascii=False, indent=2))
    print("="*80)

    return state

In [115]:
run_state_machine_verbose(question)


USER QUESTION:
State Machine 실행 모델로 Plan-Then-Execute를 돌린다는 게 무슨 의미인지 예시로 설명해줘.

---------------------------- PHASE: PLAN ----------------------------
Plan ready. Steps:
  [0] wiki :: State machine
  [1] wiki :: Plan-then-execute model
  [2] final_answer :: A State Machine is a computational model used to design algorithms that can be in one of a finite number of states at any given time. A Plan-Then-Execute model involves creating a detailed plan before executing it. When combined, this means that the system first plans out the sequence of states and transitions needed to achieve a goal, and then executes this sequence. For example, in a robotic system, the robot might plan a path (sequence of states) to navigate a room and then execute this path step-by-step, transitioning through each state as planned.

---------------------------- PHASE: EXECUTE ----------------------------
Executed 3 steps. (Next: VALIDATE)

---------------------------- PHASE: VALIDATE ---------------------------

{'user_question': 'State Machine 실행 모델로 Plan-Then-Execute를 돌린다는 게 무슨 의미인지 예시로 설명해줘.\n\n[Validator Hint]\nPlan-Then-Execute 모델에 대한 올바른 정보를 검색하여 제공해야 합니다. 이 모델은 계획을 세운 후 실행하는 방식으로, 예를 들어 로봇이 방을 탐색하기 위해 경로를 계획하고 그 경로를 단계별로 실행하는 방식으로 설명할 수 있습니다.',
 'phase': 'DONE',
 'error': None,
 'trace': [{'phase': 'PLAN',
   'ok': True,
   'used_replan_planner': False,
   'steps': [{'tool': 'wiki', 'input': 'State machine'},
    {'tool': 'wiki', 'input': 'Plan-then-execute model'},
    {'tool': 'final_answer',
     'input': 'A State Machine is a computational model used to design algorithms that can be in one of a finite number of states at any given time. A Plan-Then-Execute model involves creating a detailed plan before executing it. When combined, this means that the system first plans out the sequence of states and transitions needed to achieve a goal, and then executes this sequence. For example, in a robotic system, the robot might plan a path (sequence of states) to navigate a room and then exec

## ㄴ 3. Graph Execution Pattern
> Agent는 더 이상 **Loop**가 아닌 **Directed Graph** 임



```javascript
[Plan]
  ↓
[Search] → [Analyze]
  ↓
[Write]
  ↓
[Validate]
```



## ㄴ 4. Role-Specialized Multi-Agent Pattern


*   **System**이 **역할**을 **정의**
*   **LLM**은 **Role Function**으로 **호출**





```javascript
ResearchAgent = LLM(prompt=X)
WriterAgent   = LLM(prompt=Y)
CriticAgent   = LLM(prompt=Z)
```



## ㄴ 5. Deterministic Guardrail Pattern

**Post-ReAct에서는 반드시 아래 항목이 존재**
*   Validation Step
*   Structured Output
*   JSON Contracts
*   Retry Policy

